## Intégrations

Dans notre exemple précédent, nous avons travaillé avec des vecteurs bag-of-words de haute dimension de longueur `vocab_size`, et nous convertissions explicitement des vecteurs de représentation positionnelle de basse dimension en représentation clairsemée one-hot. Cette représentation one-hot n'est pas efficace en termes de mémoire, de plus, chaque mot est traité indépendamment des autres, c'est-à-dire que les vecteurs encodés en one-hot n'expriment aucune similarité sémantique entre les mots.

Dans cette unité, nous continuerons à explorer le jeu de données **News AG**. Pour commencer, chargeons les données et récupérons quelques définitions du notebook précédent.


In [1]:
import torch
import torchtext
import numpy as np
from torchnlp import *
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_size = len(vocab)
print("Vocab size = ",vocab_size)

Loading dataset...


d:\WORK\ai-for-beginners\5-NLP\14-Embeddings\data\train.csv: 29.5MB [00:01, 18.8MB/s]                            
d:\WORK\ai-for-beginners\5-NLP\14-Embeddings\data\test.csv: 1.86MB [00:00, 11.2MB/s]                          


Building vocab...
Vocab size =  95812


## Qu'est-ce qu'un embedding ?

L'idée de l'**embedding** est de représenter les mots par des vecteurs denses de dimension inférieure, qui reflètent d'une certaine manière le sens sémantique d'un mot. Nous discuterons plus tard de la manière de construire des embeddings de mots significatifs, mais pour l'instant, considérons simplement les embeddings comme un moyen de réduire la dimensionnalité d'un vecteur de mots.

Ainsi, une couche d'embedding prendrait un mot en entrée et produirait un vecteur de sortie de taille `embedding_size` spécifiée. En un sens, cela ressemble beaucoup à une couche `Linear`, mais au lieu de prendre un vecteur encodé en one-hot, elle pourra prendre un numéro de mot en entrée.

En utilisant une couche d'embedding comme première couche de notre réseau, nous pouvons passer du modèle bag-of-words au modèle **embedding bag**, où nous convertissons d'abord chaque mot de notre texte en son embedding correspondant, puis nous calculons une fonction d'agrégation sur tous ces embeddings, comme `sum`, `average` ou `max`.

![Image montrant un classificateur avec embedding pour une séquence de cinq mots.](../../../../../lessons/5-NLP/14-Embeddings/images/embedding-classifier-example.png)

Notre réseau de neurones classificateur commencera par une couche d'embedding, suivie d'une couche d'agrégation, puis d'un classificateur linéaire au-dessus :


In [2]:
class EmbedClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.fc = torch.nn.Linear(embed_dim, num_class)

    def forward(self, x):
        x = self.embedding(x)
        x = torch.mean(x,dim=1)
        return self.fc(x)

### Gérer la taille variable des séquences

En raison de cette architecture, les minibatches pour notre réseau devront être créés d'une certaine manière. Dans l'unité précédente, en utilisant le sac de mots (BoW), tous les tenseurs BoW dans un minibatch avaient une taille égale à `vocab_size`, indépendamment de la longueur réelle de notre séquence de texte. Une fois que nous passons aux embeddings de mots, nous nous retrouvons avec un nombre variable de mots dans chaque échantillon de texte, et lors de la combinaison de ces échantillons en minibatches, nous devrons appliquer un certain remplissage.

Cela peut être fait en utilisant la même technique qui consiste à fournir une fonction `collate_fn` à la source de données :


In [3]:
def padify(b):
    # b is the list of tuples of length batch_size
    #   - first element of a tuple = label, 
    #   - second = feature (text sequence)
    # build vectorized sequence
    v = [encode(x[1]) for x in b]
    # first, compute max length of a sequence in this minibatch
    l = max(map(len,v))
    return ( # tuple of two tensors - labels and features
        torch.LongTensor([t[0]-1 for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v])
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=padify, shuffle=True)

### Entraîner le classificateur d'embedding

Maintenant que nous avons défini un dataloader approprié, nous pouvons entraîner le modèle en utilisant la fonction d'entraînement que nous avons définie dans l'unité précédente :


In [4]:
net = EmbedClassifier(vocab_size,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=1, epoch_size=25000)

3200: acc=0.6415625
6400: acc=0.6865625
9600: acc=0.7103125
12800: acc=0.726953125
16000: acc=0.739375
19200: acc=0.75046875
22400: acc=0.7572321428571429


(0.889799795315499, 0.7623160588611644)

> **Note** : Nous n'entraînons ici que sur 25 000 enregistrements (moins d'une époque complète) pour gagner du temps, mais vous pouvez continuer l'entraînement, écrire une fonction pour entraîner sur plusieurs époques, et expérimenter avec le paramètre de taux d'apprentissage pour atteindre une meilleure précision. Vous devriez pouvoir atteindre une précision d'environ 90 %.


### Couche EmbeddingBag et Représentation de Séquences de Longueur Variable

Dans l'architecture précédente, nous devions compléter toutes les séquences pour qu'elles aient la même longueur afin de les intégrer dans un minibatch. Ce n'est pas la manière la plus efficace de représenter des séquences de longueur variable - une autre approche consiste à utiliser un vecteur **offset**, qui contient les décalages de toutes les séquences stockées dans un grand vecteur unique.

![Image montrant une représentation de séquence avec décalage](../../../../../lessons/5-NLP/14-Embeddings/images/offset-sequence-representation.png)

> **Note** : Sur l'image ci-dessus, nous montrons une séquence de caractères, mais dans notre exemple, nous travaillons avec des séquences de mots. Cependant, le principe général de représentation des séquences avec un vecteur de décalage reste le même.

Pour travailler avec la représentation par décalage, nous utilisons la couche [`EmbeddingBag`](https://pytorch.org/docs/stable/generated/torch.nn.EmbeddingBag.html). Elle est similaire à `Embedding`, mais elle prend un vecteur de contenu et un vecteur de décalage en entrée, et inclut également une couche de moyennage, qui peut être `mean`, `sum` ou `max`.

Voici un réseau modifié qui utilise `EmbeddingBag` :


In [5]:
class EmbedClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.EmbeddingBag(vocab_size, embed_dim)
        self.fc = torch.nn.Linear(embed_dim, num_class)

    def forward(self, text, off):
        x = self.embedding(text, off)
        return self.fc(x)

Pour préparer le jeu de données pour l'entraînement, nous devons fournir une fonction de conversion qui préparera le vecteur de décalage :


In [6]:
def offsetify(b):
    # first, compute data tensor from all sequences
    x = [torch.tensor(encode(t[1])) for t in b]
    # now, compute the offsets by accumulating the tensor of sequence lengths
    o = [0] + [len(t) for t in x]
    o = torch.tensor(o[:-1]).cumsum(dim=0)
    return ( 
        torch.LongTensor([t[0]-1 for t in b]), # labels
        torch.cat(x), # text 
        o
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=offsetify, shuffle=True)

Notez que, contrairement à tous les exemples précédents, notre réseau accepte désormais deux paramètres : le vecteur de données et le vecteur de décalage, qui sont de tailles différentes. De même, notre chargeur de données nous fournit également 3 valeurs au lieu de 2 : les vecteurs de texte et de décalage sont fournis comme caractéristiques. Par conséquent, nous devons légèrement ajuster notre fonction d'entraînement pour en tenir compte :


In [7]:
net = EmbedClassifier(vocab_size,32,len(classes)).to(device)

def train_epoch_emb(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.CrossEntropyLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    loss_fn = loss_fn.to(device)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,text,off in dataloader:
        optimizer.zero_grad()
        labels,text,off = labels.to(device), text.to(device), off.to(device)
        out = net(text, off)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count


train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6153125
6400: acc=0.6615625
9600: acc=0.6932291666666667
12800: acc=0.715078125
16000: acc=0.7270625
19200: acc=0.7382291666666667
22400: acc=0.7486160714285715


(22.771553103007037, 0.7551983365323096)

## Intégrations Sémantiques : Word2Vec

Dans notre exemple précédent, la couche d'intégration du modèle a appris à mapper des mots à une représentation vectorielle, mais cette représentation n'avait pas beaucoup de signification sémantique. Ce serait intéressant d'apprendre une telle représentation vectorielle où des mots similaires ou des synonymes correspondraient à des vecteurs proches les uns des autres en termes de distance vectorielle (par exemple, distance euclidienne).

Pour cela, nous devons pré-entraîner notre modèle d'intégration sur une grande collection de textes d'une manière spécifique. L'une des premières méthodes pour entraîner des intégrations sémantiques s'appelle [Word2Vec](https://en.wikipedia.org/wiki/Word2vec). Elle repose sur deux principales architectures utilisées pour produire une représentation distribuée des mots :

 - **Sac de mots continu** (CBoW) — dans cette architecture, nous entraînons le modèle à prédire un mot à partir du contexte environnant. Étant donné le ngram $(W_{-2},W_{-1},W_0,W_1,W_2)$, l'objectif du modèle est de prédire $W_0$ à partir de $(W_{-2},W_{-1},W_1,W_2)$.
 - **Skip-gram continu** est l'opposé du CBoW. Le modèle utilise une fenêtre de mots contextuels environnants pour prédire le mot actuel.

CBoW est plus rapide, tandis que skip-gram est plus lent, mais il représente mieux les mots peu fréquents.

![Image montrant les algorithmes CBoW et Skip-Gram pour convertir des mots en vecteurs.](../../../../../lessons/5-NLP/14-Embeddings/images/example-algorithms-for-converting-words-to-vectors.png)

Pour expérimenter avec l'intégration Word2Vec pré-entraînée sur le jeu de données Google News, nous pouvons utiliser la bibliothèque **gensim**. Ci-dessous, nous trouvons les mots les plus similaires à 'neural'.

> **Note :** Lorsque vous créez des vecteurs de mots pour la première fois, leur téléchargement peut prendre un certain temps !


In [8]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

In [9]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

neuronal -> 0.7804799675941467
neurons -> 0.7326500415802002
neural_circuits -> 0.7252851724624634
neuron -> 0.7174385190010071
cortical -> 0.6941086649894714
brain_circuitry -> 0.6923246383666992
synaptic -> 0.6699118614196777
neural_circuitry -> 0.6638563275337219
neurochemical -> 0.6555314064025879
neuronal_activity -> 0.6531826257705688


Nous pouvons également calculer des embeddings de vecteurs à partir du mot, à utiliser dans l'entraînement du modèle de classification (nous montrons uniquement les 20 premiers composants du vecteur pour plus de clarté) :


In [10]:
w2v.word_vec('play')[:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

La grande chose à propos des embeddings sémantiques est que vous pouvez manipuler l'encodage vectoriel pour changer la sémantique. Par exemple, nous pouvons demander de trouver un mot dont la représentation vectorielle serait aussi proche que possible des mots *roi* et *femme*, et aussi éloignée que possible du mot *homme* :


In [10]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118192911148071)

Les modèles CBoW et Skip-Grams sont des embeddings dits "prédictifs", car ils ne prennent en compte que les contextes locaux. Word2Vec ne tire pas parti du contexte global.

**FastText** s'appuie sur Word2Vec en apprenant des représentations vectorielles pour chaque mot ainsi que pour les n-grammes de caractères présents dans chaque mot. Les valeurs de ces représentations sont ensuite moyennées en un seul vecteur à chaque étape d'entraînement. Bien que cela ajoute beaucoup de calculs supplémentaires lors de la pré-formation, cela permet aux embeddings de mots d'intégrer des informations sur les sous-mots.

Une autre méthode, **GloVe**, exploite l'idée de matrice de cooccurrence et utilise des méthodes neuronales pour décomposer cette matrice en vecteurs de mots plus expressifs et non linéaires.

Vous pouvez expérimenter avec cet exemple en changeant les embeddings pour FastText et GloVe, car gensim prend en charge plusieurs modèles d'embeddings de mots différents.


## Utilisation des embeddings pré-entraînés dans PyTorch

Nous pouvons modifier l'exemple ci-dessus pour pré-remplir la matrice de notre couche d'embedding avec des embeddings sémantiques, comme Word2Vec. Il faut tenir compte du fait que les vocabulaires des embeddings pré-entraînés et de notre corpus de texte ne correspondront probablement pas, donc nous initialiserons les poids des mots manquants avec des valeurs aléatoires :


In [11]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

net = EmbedClassifier(vocab_size,embed_size,len(classes))

print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab.get_itos()):
    try:
        net.embedding.weight[i].data = torch.tensor(w2v.get_vector(w))
        found+=1
    except:
        net.embedding.weight[i].data = torch.normal(0.0,1.0,(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")
net = net.to(device)

Embedding size: 300
Populating matrix, this will take some time...Done, found 41080 words, 54732 words missing


Maintenant, entraînons notre modèle. Notez que le temps nécessaire pour entraîner le modèle est significativement plus long que dans l'exemple précédent, en raison de la taille plus importante de la couche d'embedding, et donc d'un nombre de paramètres beaucoup plus élevé. De plus, à cause de cela, nous pourrions avoir besoin d'entraîner notre modèle sur davantage d'exemples si nous voulons éviter le surapprentissage.


In [12]:
train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6359375
6400: acc=0.68109375
9600: acc=0.7067708333333333
12800: acc=0.723671875
16000: acc=0.73625
19200: acc=0.7463541666666667
22400: acc=0.7560714285714286


(214.1013875559821, 0.7626759436980166)

Dans notre cas, nous ne constatons pas une augmentation significative de la précision, ce qui est probablement dû à des vocabulaires très différents.  
Pour surmonter le problème des vocabulaires différents, nous pouvons utiliser l'une des solutions suivantes :  
* Réentraîner le modèle word2vec sur notre vocabulaire  
* Charger notre jeu de données avec le vocabulaire du modèle word2vec pré-entraîné. Le vocabulaire utilisé pour charger le jeu de données peut être spécifié lors du chargement.  

La deuxième approche semble plus simple, surtout parce que le framework `torchtext` de PyTorch contient un support intégré pour les embeddings. Nous pouvons, par exemple, instancier un vocabulaire basé sur GloVe de la manière suivante :  


In [14]:
vocab = torchtext.vocab.GloVe(name='6B', dim=50)

100%|█████████▉| 399999/400000 [00:15<00:00, 25411.14it/s]


Le vocabulaire chargé propose les opérations de base suivantes :  
* Le dictionnaire `vocab.stoi` nous permet de convertir un mot en son index dans le dictionnaire.  
* `vocab.itos` fait l'inverse - il convertit un numéro en mot.  
* `vocab.vectors` est le tableau des vecteurs d'embedding, donc pour obtenir l'embedding d'un mot `s`, nous devons utiliser `vocab.vectors[vocab.stoi[s]]`.  

Voici un exemple de manipulation des embeddings pour démontrer l'équation **kind-man+woman = queen** (j'ai dû ajuster légèrement le coefficient pour que cela fonctionne) :  


In [15]:
# get the vector corresponding to kind-man+woman
qvec = vocab.vectors[vocab.stoi['king']]-vocab.vectors[vocab.stoi['man']]+1.3*vocab.vectors[vocab.stoi['woman']]
# find the index of the closest embedding vector 
d = torch.sum((vocab.vectors-qvec)**2,dim=1)
min_idx = torch.argmin(d)
# find the corresponding word
vocab.itos[min_idx]

'queen'

Pour entraîner le classificateur en utilisant ces embeddings, nous devons d'abord encoder notre ensemble de données en utilisant le vocabulaire GloVe :


In [16]:
def offsetify(b):
    # first, compute data tensor from all sequences
    x = [torch.tensor(encode(t[1],voc=vocab)) for t in b] # pass the instance of vocab to encode function!
    # now, compute the offsets by accumulating the tensor of sequence lengths
    o = [0] + [len(t) for t in x]
    o = torch.tensor(o[:-1]).cumsum(dim=0)
    return ( 
        torch.LongTensor([t[0]-1 for t in b]), # labels
        torch.cat(x), # text 
        o
    )

Comme nous l'avons vu ci-dessus, toutes les représentations vectorielles sont stockées dans la matrice `vocab.vectors`. Cela rend extrêmement facile de charger ces poids dans les poids de la couche d'embedding en utilisant une simple copie :


In [17]:
net = EmbedClassifier(len(vocab),len(vocab.vectors[0]),len(classes))
net.embedding.weight.data = vocab.vectors
net = net.to(device)

In [18]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=offsetify, shuffle=True)
train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6271875
6400: acc=0.68078125
9600: acc=0.7030208333333333
12800: acc=0.71984375
16000: acc=0.7346875
19200: acc=0.7455729166666667
22400: acc=0.7529464285714286


(35.53972978646833, 0.7575175943698017)

Une des raisons pour lesquelles nous ne constatons pas d'augmentation significative de la précision est le fait que certains mots de notre ensemble de données sont absents du vocabulaire pré-entraîné de GloVe, et sont donc essentiellement ignorés. Pour surmonter ce problème, nous pouvons entraîner nos propres embeddings sur notre ensemble de données.


## Contextual Embeddings

Une des principales limites des représentations d'embeddings préentraînés traditionnels comme Word2Vec est le problème de la désambiguïsation des sens des mots. Bien que les embeddings préentraînés puissent capturer une partie du sens des mots dans un contexte donné, tous les sens possibles d'un mot sont encodés dans le même embedding. Cela peut poser des problèmes dans les modèles en aval, car de nombreux mots, comme le mot "play", ont des significations différentes selon le contexte dans lequel ils sont utilisés.

Par exemple, le mot "play" dans ces deux phrases a des significations très différentes :
- Je suis allé voir une **pièce** au théâtre.
- John veut **jouer** avec ses amis.

Les embeddings préentraînés ci-dessus représentent ces deux significations du mot "play" dans le même embedding. Pour surmonter cette limitation, nous devons construire des embeddings basés sur le **modèle de langage**, qui est entraîné sur un large corpus de texte et *comprend* comment les mots peuvent être assemblés dans différents contextes. La discussion sur les embeddings contextuels dépasse le cadre de ce tutoriel, mais nous y reviendrons en abordant les modèles de langage dans la prochaine unité.



---

**Avertissement** :  
Ce document a été traduit à l'aide du service de traduction automatique [Co-op Translator](https://github.com/Azure/co-op-translator). Bien que nous nous efforcions d'assurer l'exactitude, veuillez noter que les traductions automatisées peuvent contenir des erreurs ou des inexactitudes. Le document original dans sa langue d'origine doit être considéré comme la source faisant autorité. Pour des informations critiques, il est recommandé de recourir à une traduction professionnelle réalisée par un humain. Nous déclinons toute responsabilité en cas de malentendus ou d'interprétations erronées résultant de l'utilisation de cette traduction.
